# Access EERIE data via eerie.cloud.dkrz.de

EERIE data from ICON and IFS-FESOM2 is made accessible via a fast lane web service under eerie.cloud.dkrz.de. The web service reflects a live view on all EERIE experiments run and saved at DKRZ. 

All available datasets are listed [here](https://eerie.cloud.dkrz.de/datasets). For each dataset, both a xarray dataset view, e.g. for [this FESOM data set](https://eerie.cloud.dkrz.de/datasets/IFS-FESOM2-SR_eerie-control-1950_FESOM_025_daily/) as well as a zarr endpoint is provided which can be directly opened with xarray. This is enabled by the python package [xpublish](https://github.com/xpublish-community/xpublish).

Xpublish allows to add plugins to the server as well as server-side processing. At DKRZ, we run the intake plugin which generates an intake catalog [(download link)](https://eerie.cloud.dkrz.de/intake.yaml) for all datasets on the server. Additionally, for all datasets we offer a precision-trimmed version which reduces the required storage for saving the dataset at another location while not affecting results of analysis significantly.

To access the data and run this notebook, you need a kernel / environment with the following packages installed:

```
- intake-xarray
- intake
- zarr
- dask
- hvplot # for plotting
- aiohttp
- requests
```

Before we do anything with the data, we have to set up a dask client. The eerie.cloud server allows 4 concurrent threads per user because the application on the server is very memory intensive. We therefore open a client with 4 threads:

In [1]:
from distributed import Client
client=Client(n_workers=2,threads_per_worker=2)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 2
Total threads: 4,Total memory: 195.31 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36017,Workers: 2
Dashboard: /proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 195.31 GiB
Comm: tcp://127.0.0.1:39441,Total threads: 2
Dashboard: /proxy/44757/status,Memory: 97.66 GiB
Nanny: tcp://127.0.0.1:39487,


## Using intake for loading

See [this notebook](https://github.com/eerie-project/EERIE_hackathon_2023/blob/main/COMMON/eerie_data-access_dkrz-disk.ipynb) how to use intake to browse and load data sources.

In [5]:
import intake
eerie_cat=intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")
cat=eerie_cat["dkrz.cloud"]

We can list all datasets available in the catalog with `list`:

In [6]:
all_dkrz_datasets=list(cat)
print(all_dkrz_datasets[0])

ICON-ESM-ER_eerie-control-1950_atm_2d_1d_max_12bit


- datasets with suffix *'_12bit'* are lossy compressed on the fly with bitrounding to reduce traffic
- datasets with suffix *'_remap025'* are retrieved via a kerchunk so that eerie.cloud is only used as a data server

We can get the asset of a dataset (i.e. the actual file that can be opened and described as a catalog) by using it as a key in the `cat` dictionary:

In [10]:
dset="ICON-ESM-ER_eerie-control-1950_atm_2d_1mth_mean_remap025"
icon_control_atm_2d_1mth_mean=cat[dset]

In [ ]:
list(icon_control_atm_2d_1mth_mean)

## Opening the dataset

Each intake-xarray asset can be opened with the `to_dask` function:

In [12]:
ds=icon_control_atm_2d_1mth_mean[dset+"-zarr"].to_dask()

In [13]:
ds

<xarray.Dataset>
Dimensions:   (time: 153, lat: 721, lon: 1440, height: 1, height_2: 1)
Coordinates:
  * height    (height) float64 2.0
  * height_2  (height_2) float64 10.0
  * lat       (lat) float64 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * lon       (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * time      (time) datetime64[ns] 2002-02-28T23:59:00 ... 2014-11-30T23:59:00
Data variables: (12/32)
    clivi     (time, lat, lon) float32 dask.array<chunksize=(1, 182, 1440), meta=np.ndarray>
    cllvi     (time, lat, lon) float32 dask.array<chunksize=(1, 182, 1440), meta=np.ndarray>
    clt       (time, lat, lon) float32 dask.array<chunksize=(1, 182, 1440), meta=np.ndarray>
    dew2      (time, height, lat, lon) float32 dask.array<chunksize=(1, 1, 182, 1440), meta=np.ndarray>
    evspsbl   (time, lat, lon) float32 dask.array<chunksize=(1, 182, 1440), meta=np.ndarray>
    hfls      (time, lat, lon) float32 dask.array<chunksize=(1, 182, 1440), meta=np.ndarray>
    ...        ...
    tasmin    (time, height, lat, lon) float32 dask.array<chunksize=(1, 1, 182, 1440), meta=np.ndarray>
    tauu      (time, lat, lon) float32 dask.array<chunksize=(1, 182, 1440), meta=np.ndarray>
    tauv      (time, lat, lon) float32 dask.array<chunksize=(1, 182, 1440), meta=np.ndarray>
    ts        (time, lat, lon) float32 dask.array<chunksize=(1, 182, 1440), meta=np.ndarray>
    uas       (time, height_2, lat, lon) float32 dask.array<chunksize=(1, 1, 182, 1440), meta=np.ndarray>
    vas       (time, height_2, lat, lon) float32 dask.array<chunksize=(1, 1, 182, 1440), meta=np.ndarray>
Attributes: (12/27)
    CDI:                       Climate Data Interface version 2.2.4 (https://...
    CDO:                       Climate Data Operators version 2.2.2 (https://...
    Conventions:               CF-1.6
    DOKU_License:              CC BY 4.0
    DOKU_Name:                 EERIE ICON Control run
    DOKU_authors:              Putrasahan, D.; Kröger, J.; Wachsmann, F.
    ...                        ...
    references:                see MPIM/DWD publications
    simulation_id:             erc1011
    source:                    git@gitlab.dkrz.de:icon/icon-mpim.git@45022778...
    source_id:                 ICON-ESM
    time_reduction:            mean
    title:                     ICON simulation

## Calculate, load and plot an example

In the following, we 

1. select a variable
2. calculate a field mean over the globe
3. calculate a yearly mean
4. plot the data

In [ ]:
var="tas"
tas_fldmean=ds[var].isel(time=range(0,3)).mean(dim="ncells")
tas_fldmean

Note that a field mean is not a global mean. For the latter, we need to weight by the cell area which requires the grid of the data. The grid is available in another dataset.

The same is valid for yearly means. We actually would need to weight the monthly means with the days of the month. However, this is just a quick example:

In [ ]:
tas_yearmean=tas_fldmean.groupby("time.year").mean()
tas_yearmean

The hvplot library can be used to generate interactive plots.

In [ ]:
import hvplot.xarray
tas_yearmean.squeeze().hvplot.line()

You can save the file to any location where you have access with:

In [ ]:
#tas_yearmean.to_zarr(ZARRDIR)